# Notebook contains summary of adversarial robustness of different model

In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
from datetime import datetime as dt
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.attacks import fgsm, pgd
from utils.utils import plot_images
from models.resnet import *
from models.wide_resnet import WideResNet, SparseWideResNet

1.7.0
True


In [3]:
from datetime import datetime as dt

def clean_acc(model, loader):
    clean_correct = 0.
    model.eval()
    with torch.no_grad():
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)  
            clean_correct += (model(X).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        acc = round(clean_correct / len(loader.dataset), 4)
        print(f"clean accuracy: {acc}")
    return acc

def adv_acc(model, loader, loss_fn, attack, attack_params):
    model.eval()
    adv_correct = [0] * len(attack_params)
    for i, (name, params) in enumerate(attack_params):
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)
            noise = attack(model, X, y, loss_fn, **params) 
            adv_correct[i] += (model(X+noise).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        print(f"name: {name} accuracy: {(adv_correct[i] / len(loader.dataset)):.4f}")
    return [round(a/len(loader.dataset), 4) for a in adv_correct]


def print_table(models, attacks, clean, adv):
    d = {'model': models, 'clean image': clean}
    for i, (name, p) in enumerate(attacks):
        d[name] = [adv[j][i] for j, _ in enumerate(adv)]

    return pd.DataFrame(data=d)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [5]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

ds = CIFAR10('../data', train=False, target_transform=None, download=True, transform=test_transform)
test_loader = DataLoader(ds, 500, shuffle=True)

Files already downloaded and verified


In [6]:
def create_attack(eps, alpha, num_iter):
    d = {
        "epsilon": eps,
        "alpha": alpha,
        "num_iter": num_iter
    }
    return (f"e={eps:.4f},a={alpha:.4f},i={num_iter}", d)
    

attack1 = create_attack(eps=8/255, alpha=2/255, num_iter=20)
attack2 = create_attack(eps=4/255, alpha=2/255, num_iter=20)

params = [attack1, attack2]

# ResNet18 vs Sparse ResNet 18

In [7]:
clean, adv = [], []

In [8]:
model = resnet18().to(device)
model.load_state_dict(torch.load("../saved/resnet18.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

clean.append(acc)
adv.append(acc_adv)

clean accuracy: 0.9272                            
name: e=0.0314,a=0.0078,i=20 accuracy: 0.0000     
name: e=0.0157,a=0.0078,i=20 accuracy: 0.0006     


In [9]:
model = sparse_resnet18().to(device)
model.load_state_dict(torch.load("../saved/sparse_resnet18.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

clean.append(acc)
adv.append(acc_adv)

clean accuracy: 0.9082                            
name: e=0.0314,a=0.0078,i=20 accuracy: 0.0000     
name: e=0.0157,a=0.0078,i=20 accuracy: 0.0042     


In [10]:
model = resnet18().to(device)
model.load_state_dict(torch.load("../saved/resnet18_AT.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

clean.append(acc)
adv.append(acc_adv)

clean accuracy: 0.7906                            
name: e=0.0314,a=0.0078,i=20 accuracy: 0.4765     
name: e=0.0157,a=0.0078,i=20 accuracy: 0.6515     


In [12]:
model = sparse_resnet18().to(device)
model.load_state_dict(torch.load("../saved/sparse_resnet18_AT.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

clean.append(acc)
adv.append(acc_adv)

clean accuracy: 0.6714                            
name: e=0.0314,a=0.0078,i=20 accuracy: 0.3475     
name: e=0.0157,a=0.0078,i=20 accuracy: 0.5142     


In [13]:
models_names = ["ResNet18", "Sparse ResNet18", "ResNet18 AT", "Sparse ResNet18 AT"]
resnets = print_table(models_names, params, clean, adv)
resnets

,model,clean image,"e=0.0314,a=0.0078,i=20","e=0.0157,a=0.0078,i=20"
0,ResNet18,0.9272,0.0000,0.0006
1,Sparse ResNet18,0.9082,0.0000,0.0042
2,ResNet18 AT,0.7906,0.4765,0.6515
3,Sparse ResNet18 AT,0.6714,0.3475,0.5142
